In [2]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 30 kB/s 
     |████████████████████████████████| 198 kB 58.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=549dc54270313bc0497c8d54e6c5fbdc166a5911fbb264f7b0f3a13d97733761
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark =SparkSession.builder.getOrCreate()

In [54]:
#user defined functions----
import pandas
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series:pd.Series) -> pd.Series:
  return series + 1

df.select(pandas_plus_one(df.no)).show() 

+-------------------+
|pandas_plus_one(no)|
+-------------------+
|                  2|
|                  3|
|                  4|
|                 12|
|                  3|
|                  4|
|                 23|
|                  3|
|                  4|
+-------------------+



In [55]:
#user defined functions----
def pandas_filter_func(iterator):
  for pandas_df in iterator:
    yield pandas_df[pandas_df.no == 8.0]
df.mapInPandas(pandas_filter_func,schema=df.schema).show()


+-----+-----+---+----+
|color|fruit| no|rate|
+-----+-----+---+----+
+-----+-----+---+----+



In [53]:
#creating dataframe---
df=spark.createDataFrame([
   ['red','banana',1,10],['blue','banana',2,20],['yello','grape',3,120],
   ['pink','mango',11,100],['white','banana',2,20],['yello','grape',3,120],
   ['voilet','pi',22,110],['blue','banana',2,20],['yello','grape',3,120]],schema=('color string,fruit string,no long,rate long')                       
)
df.show()

+------+------+---+----+
| color| fruit| no|rate|
+------+------+---+----+
|   red|banana|  1|  10|
|  blue|banana|  2|  20|
| yello| grape|  3| 120|
|  pink| mango| 11| 100|
| white|banana|  2|  20|
| yello| grape|  3| 120|
|voilet|    pi| 22| 110|
|  blue|banana|  2|  20|
| yello| grape|  3| 120|
+------+------+---+----+



In [21]:
#group by command with avg func---
df.groupBy('color').avg().show()

+------+-------+---------+
| color|avg(no)|avg(rate)|
+------+-------+---------+
| yello|    3.0|    120.0|
|  pink|   11.0|    100.0|
|   red|    1.0|     10.0|
|  blue|    2.0|     20.0|
| white|    2.0|     20.0|
|voilet|   22.0|    110.0|
+------+-------+---------+



In [20]:
#group by command with min func---
df.groupBy('rate').min().show()

+----+-------+---------+
|rate|min(no)|min(rate)|
+----+-------+---------+
|  10|      1|       10|
| 120|      3|      120|
| 100|     11|      100|
|  20|      2|       20|
| 110|     22|      110|
+----+-------+---------+



In [30]:
#joining operation---
df1=spark.createDataFrame(
    [(2000002,1,1.0),(300002,2,2.0),(400000,3,3.0)],('time','id','v1'))

df2=spark.createDataFrame(
    [(2000002,1,1.0),(300002,2,2.0),(400000,3,3.0)],('time','id','v2'))
def asof_join(l,r):
  return pd.merge_asof(l,r,on='time',by='id')

df1.groupBy('id').cogroup(df2.groupBy('id')).applyInPandas(
    asof_join,schema='time int,id int,v1 double,v2 double'
).show()

+-------+---+---+---+
|   time| id| v1| v2|
+-------+---+---+---+
|2000002|  1|1.0|1.0|
| 300002|  2|2.0|2.0|
| 400000|  3|3.0|3.0|
+-------+---+---+---+



In [34]:
#to read the file----
spark.read.csv('/content/sample_data/california_housing_test.csv',header=True).show()

+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-122.050000|37.370000|         27.000000|3885.000000|    661.000000|1537.000000| 606.000000|     6.608500|     344700.000000|
|-118.300000|34.260000|         43.000000|1510.000000|    310.000000| 809.000000| 277.000000|     3.599000|     176500.000000|
|-117.810000|33.780000|         27.000000|3589.000000|    507.000000|1484.000000| 495.000000|     5.793400|     270500.000000|
|-118.360000|33.820000|         28.000000|  67.000000|     15.000000|  49.000000|  11.000000|     6.135900|     330000.000000|
|-119.670000|36.330000|         19.000000|1241.000000|    244.000000| 850.000000| 237.000000|     2.937500|    

In [39]:
#for creating  the csv file from dataframe--
df.write.csv('foo.csv',header= True)

In [46]:
#for creating  the orc file from dataframe--
df.write.orc('abhi.orc')


In [47]:
#for reading  the orc file --
spark.read.orc('abhi.orc').show()

+-----+------+------+----------+-------------------+
|apple|banana|     c|         d|                  e|
+-----+------+------+----------+-------------------+
|    2|   3.0|hello1|2021-01-04|2021-02-12 00:00:00|
|    3|   4.0|hello2|2021-02-05|2021-02-11 01:00:00|
|    1|   2.0| hello|2021-02-04|2021-02-12 01:00:00|
+-----+------+------+----------+-------------------+



In [50]:
#for creating  the parquet file from dataframe--
df.write.parquet('niki1.parquet')

In [52]:
#for reading  the parquet file----
spark.read.parquet('niki1.parquet').show()

+-----+------+------+----------+-------------------+
|apple|banana|     c|         d|                  e|
+-----+------+------+----------+-------------------+
|    1|   2.0| hello|2021-02-04|2021-02-12 01:00:00|
|    2|   3.0|hello1|2021-01-04|2021-02-12 00:00:00|
|    3|   4.0|hello2|2021-02-05|2021-02-11 01:00:00|
+-----+------+------+----------+-------------------+

